In [12]:
import tkinter as tk
from tkinter import filedialog, messagebox
import subprocess
import os
import re
import requests
import shutil

# Название модели
model_name = "RealESRGAN_x2plus"

# Путь для сохранения весов модели
weights_folder = f"weights_{model_name}"
weights_file = os.path.join(weights_folder, f"{model_name}.pth")

# Максимальный допустимый размер файла (10 МБ)
max_file_size_mb = 1000
max_file_size_bytes = max_file_size_mb * 1024 * 1024

# Скачивание весов модели Real-ESRGAN
def download_weights():
    url = f"https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/{model_name}.pth"

    if not os.path.exists(weights_file):
        os.makedirs(weights_folder, exist_ok=True)  # Создаем директорию, если она не существует
        print("Downloading weights, please wait...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))

        with open(weights_file, 'wb') as f:
            for data in response.iter_content(1024):
                f.write(data)
        messagebox.showinfo("Download Complete", f"Weights downloaded and saved to {weights_file}")
    else:
        messagebox.showinfo("Info", "Weights already exist. No need to download again.")

# Функция для конвертации путей с заменой на двойные прямые слэши
def normalize_path(path):
    # Заменяем одиночные обратные слэши (\) или прямые слэши (/) на двойные прямые слэши (//)
    # Игнорируем уже существующие двойные прямые слэши
    normalized_path = re.sub(r'(?<!/)/(?!/)|\\+', '//', path)
    return normalized_path

# Функция для увеличения разрешения нескольких изображений с помощью Real-ESRGAN
def upscale_images():
    img_paths = filedialog.askopenfilenames(title="Select images", filetypes=[("Image files", "*.png;*.jpg;*.jpeg"), ("All files", "*.*")])
    
    if img_paths:
        # Проверка наличия весов
        if not os.path.exists(weights_file):
            messagebox.showerror("Error", "Weights not found. Please download weights first.")
            return

        # Проверка наличия скрипта inference_realesrgan.py
        if not os.path.exists('inference_realesrgan.py'):
            messagebox.showerror("Error", "inference_realesrgan.py not found in the current directory.")
            return

        # Обработка каждого изображения
        for img_path in img_paths:
            img_path = normalize_path(img_path)  # Конвертируем путь
            
            # Генерация пути для сохранения с добавлением суффикса _out
            img_dir, img_name = os.path.split(img_path)
            img_name_no_ext, img_ext = os.path.splitext(img_name)
            output_file = normalize_path(os.path.join(img_dir, f"{img_name_no_ext}_x2_upscaled{img_ext}"))

            # Команда для запуска Real-ESRGAN
            command = f"python inference_realesrgan.py -n {model_name} -i \"{img_path}\" -o \"{img_dir}\" --outscale 2 --suffix x2_upscaled"
            print(f"Running command: {command}")
            try:
                subprocess.run(command, check=True, shell=True)
                print(f"Image has been upscaled and saved to {output_file}")

                # Проверяем размер итогового файла
                if os.path.getsize(output_file) > max_file_size_bytes:
                    # Если файл больше 10 МБ, добавляем пометку к названию
                    large_file_output = normalize_path(os.path.join(img_dir, f"{img_name_no_ext}_x2_upscaled_large{img_ext}"))
                    os.rename(output_file, large_file_output)
                    print(f"File size exceeds {max_file_size_mb} MB. Renamed to {large_file_output}")

            except subprocess.CalledProcessError as e:
                print(f"Error during subprocess: {e}")
                messagebox.showerror("Error", f"Failed to upscale image: {img_name}\nError: {e}")
        
        messagebox.showinfo("Success", "All images have been upscaled successfully.")

# # Функция для перемещения изображений из созданных папок на уровень выше и удаления папок
# def move_images_from_folders(img_dir):
#     # Проход по всем папкам в директории, где были выбраны изображения
#     for root, dirs, files in os.walk(img_dir):
#         for dir_name in dirs:
#             if '_out' in dir_name:  # Ищем папки с суффиксом _out
#                 dir_path = os.path.join(root, dir_name)
#                 files_in_dir = os.listdir(dir_path)

#                 # Проверяем, есть ли внутри один файл
#                 if len(files_in_dir) == 1:
#                     file_path = os.path.join(dir_path, files_in_dir[0])
#                     file_name, file_ext = os.path.splitext(files_in_dir[0])

#                     # Генерация нового имени с суффиксом _x2_upscaled
#                     new_file_name = f"{file_name.split('_out')[0]}_x2_upscaled{file_ext}"

#                     # Перемещаем файл на уровень выше (в родительскую директорию)
#                     try:
#                         new_path = os.path.join(root, new_file_name)
#                         shutil.move(file_path, new_path)
#                         print(f"Moved and renamed {files_in_dir[0]} to {new_file_name}")
#                     except Exception as e:
#                         print(f"Failed to move {files_in_dir[0]}: {e}")

#                     # Удаляем пустую папку
#                     try:
#                         os.rmdir(dir_path)
#                         print(f"Deleted folder: {dir_name}")
#                     except Exception as e:
#                         print(f"Failed to delete folder {dir_name}: {e}")
#                 else:
#                     print(f"Skipping folder {dir_name}, it contains more than one file.")

# Создаем окно
root = tk.Tk()
root.title("Image Upscaler with Real-ESRGAN")
root.geometry("400x300")

# Инструкции для пользователя
instructions = tk.Label(root, text="Steps to Upscale Image:", font=("Arial", 14), pady=10)
instructions.pack()

steps_frame = tk.Frame(root)
steps_frame.pack(pady=10)

# Шаги
step1 = tk.Label(steps_frame, text="1. Download Weights", font=("Arial", 12))
step1.grid(row=0, column=0, pady=5, sticky='w')
step2 = tk.Label(steps_frame, text="2. Select Images", font=("Arial", 12))
step2.grid(row=1, column=0, pady=5, sticky='w')
step3 = tk.Label(steps_frame, text="3. Upscale Images", font=("Arial", 12))
step3.grid(row=2, column=0, pady=5, sticky='w')

# Кнопка для скачивания весов
download_btn = tk.Button(root, text="Download Weights (Model)", command=download_weights, width=25, font=("Arial", 12))
download_btn.pack(pady=10)

# Кнопка для увеличения изображений
upscale_btn = tk.Button(root, text="Upscale Images", command=upscale_images, width=25, font=("Arial", 12))
upscale_btn.pack(pady=10)

# Запуск окна
root.mainloop()


Running command: python inference_realesrgan.py -n RealESRGAN_x2plus -i "C://Users//d_tru//Downloads//D//gruzi_suda//VSE_PO_MARKETPLACES//VSE_NASHE//images_for_infograph//1.png" -o "C://Users//d_tru//Downloads//D//gruzi_suda//VSE_PO_MARKETPLACES//VSE_NASHE//images_for_infograph" --outscale 2 --suffix x2_upscaled
Image has been upscaled and saved to C://Users//d_tru//Downloads//D//gruzi_suda//VSE_PO_MARKETPLACES//VSE_NASHE//images_for_infograph//1_x2_upscaled.png
Running command: python inference_realesrgan.py -n RealESRGAN_x2plus -i "C://Users//d_tru//Downloads//D//gruzi_suda//VSE_PO_MARKETPLACES//VSE_NASHE//images_for_infograph//4.png" -o "C://Users//d_tru//Downloads//D//gruzi_suda//VSE_PO_MARKETPLACES//VSE_NASHE//images_for_infograph" --outscale 2 --suffix x2_upscaled
Image has been upscaled and saved to C://Users//d_tru//Downloads//D//gruzi_suda//VSE_PO_MARKETPLACES//VSE_NASHE//images_for_infograph//4_x2_upscaled.png
Running command: python inference_realesrgan.py -n RealESRGAN_x2pl